In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

import pyAgrum.skbn as skbn

import pandas as pd
import numpy as np

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from estimation_class.estimators import estimator

In [5]:
# Model parameters
XI = np.array([-1, 1, 1])
NU = np.array([0, -1, 1, -1, 2])
BETA = np.array([0, 0.6, -0.6, 0.6])
TAU_0 = np.array([-1, -1, 1, 1])
TAU_1 = TAU_0 * -1
SIGMA = np.array([[1, 0.5, -0.5, -0.5],
                  [0.5, 1, -0.5, -0.5],
                  [-0.5, -0.5, 1, 0.5],
                  [-0.5, -0.5, 0.5, 1]], dtype=float)

def generate_lunceford(n=1000):
    # Generate data
    x3 = np.random.binomial(1, 0.2, n)
    v3 = np.random.binomial(1, (0.75 * x3 + (0.25 * (1 - x3))), n)

    # If x3=0 you have a model, if x3=1 you have another one
    x1v1x2v2_x3_0_matrix = np.random.multivariate_normal(TAU_0, SIGMA, size=n, check_valid='warn', tol=1e-8)
    x1v1x2v2_x3_1_matrix = np.random.multivariate_normal(TAU_1, SIGMA, size=n, check_valid='warn', tol=1e-8)
    x1v1x2v2_x3 = np.where(np.repeat(x3[:, np.newaxis], 4, axis=1) == 0, x1v1x2v2_x3_0_matrix, x1v1x2v2_x3_1_matrix)

    # Concatenate values
    xv = np.concatenate([x1v1x2v2_x3, np.expand_dims(x3, axis=1), np.expand_dims(v3, axis=1)], axis=1)

    # Compute e, a, and y
    x = xv[:, [0,2,4]]
    v = xv[:, [1,3,5]]
    e = np.power(1 + np.exp(- BETA[0] - x.dot(BETA[1:])), -1)
    a = np.random.binomial(1, e, n)
    y = x.dot(NU[1:-1]) + v.dot(XI) + a*NU[-1] + np.random.binomial(1, e, n) + np.random.normal(0, 1, n)

    # Create the final df
    synthetic_data_df = pd.DataFrame(np.concatenate([x, np.expand_dims(a, axis=1), v, np.expand_dims(y, axis=1)], axis=1), columns=["X1", "X2", "X3", "T", "V1", "V2", "V3", "Y"])
    synthetic_data_df["X3"] = synthetic_data_df["X3"].astype(int)
    synthetic_data_df["V3"] = synthetic_data_df["V3"].astype(int)
    synthetic_data_df["T"] = synthetic_data_df["T"].astype(int)

    return synthetic_data_df

In [6]:
df = generate_lunceford(int(1e6))
df.head()

,X1,X2,X3,T,V1,V2,V3,Y
0,0.705361,-0.636464,1,0,1.144012,-2.001472,0,-4.342260
1,-0.637608,1.013629,0,0,-0.962611,0.244434,0,1.886090
2,-0.082324,0.057769,0,0,0.161763,0.200178,0,3.068495
3,0.342855,1.412522,0,1,-1.124878,0.198501,0,4.571569
4,0.236811,-0.040290,1,1,1.408499,-0.700456,1,-0.256451


In [7]:
dag = gum.DAG()

dag.addNodes(8)

dag.addArc(0,1)
dag.addArc(0,2)
dag.addArc(0,3)
dag.addArc(0,4)
dag.addArc(0,5)
dag.addArc(0,6)
dag.addArc(0,7)

dag.addArc(1,7)
dag.addArc(2,7)
dag.addArc(3,7)
dag.addArc(4,7)
dag.addArc(5,7)
dag.addArc(6,7)

In [8]:
dag

(pyAgrum.DAG@0x60e3aeca8300) {0,1,2,3,4,5,6,7} , {5->7,3->7,1->7,0->5,0->1,0->6,0->2,4->7,2->7,0->7,0->3,6->7,0->4}

In [9]:
def getBN(# Covariate parameters
          covariate_start : int = -5.0,
          covariate_end : int = 5.0 ,
          covariate_num_split : int = 10,
          # Outcome parameters
          outcome_start = -10.0 ,
          outcome_end = 15.0 ,
          outcome_num_split = 60,
          # Other
          data : pd.DataFrame | None = None,
          add_arcs : bool = True,
          fill_distribution : bool = True) -> gum.BayesNet:
    """
    Returns Baysian Network corresponding to the model by discretising
    countinous variables with given parameters.
    """
    if data is None:
        plus = "" if fill_distribution else "+"
        bn = gum.BayesNet()
        for i in range(1,3):
            bn.add(f"X{i}{plus}[{covariate_start}:{covariate_end}:{covariate_num_split}]")
            bn.add(f"V{i}{plus}[{covariate_start}:{covariate_end}:{covariate_num_split}]")
        bn.add(f"X3[2]")
        bn.add(f"V3[2]")
        bn.add("T[2]")
        bn.add(f"Y{plus}[{outcome_start}:{outcome_end}:{outcome_num_split}]")

    else :
        disc = skbn.BNDiscretizer(defaultDiscretizationMethod="uniform",
                                  defaultNumberOfBins=covariate_num_split)
        disc.setDiscretizationParameters("X3", 'NoDiscretization', [0, 1])
        disc.setDiscretizationParameters("V3", 'NoDiscretization', [0, 1])
        disc.setDiscretizationParameters("T", 'NoDiscretization', [0, 1])
        disc.setDiscretizationParameters("Y", 'uniform', outcome_num_split)
        bn = disc.discretizedBN(data)

    if add_arcs :
        bn.beginTopologyTransformation()
        for _, name in bn:
            if name != "Y":
                bn.addArc(name, "Y")
        for X in ["X1", "X2", "X3"]:
            bn.addArc(X, "T")
        for XV in ["X1", "V1", "X2", "V2"]:
            bn.addArc("X3", XV)
        bn.addArc("X3", "V3")
        bn.endTopologyTransformation()

    if add_arcs and fill_distribution:
        bn.cpt("X3").fillWith([0.8, 0.2])
        bn.cpt("V3")[:] = [[0.75, 0.25], [0.25, 0.75]]
        for XV in ["X", "V"]:
            bn.cpt(f"{XV}1").fillFromDistribution(norm, loc="2*X3-1", scale=1)
            bn.cpt(f"{XV}2").fillFromDistribution(norm, loc="1-2*X3", scale=1)
        bn.cpt("T").fillFromDistribution(logistic, loc="-0.6*X1+0.6*X2-0.6*X3", scale=1)
        bn.cpt("Y").fillFromDistribution(norm, loc="-X1+X2-X3+2*T-V1+V2+V3", scale=1)

    return bn

In [10]:
bn = getBN(data=df, fill_distribution=False)
bn

(pyAgrum.BayesNet<double>@0x60e3aed424f0) BN{nodes: 8, arcs: 15, domainSize: 10^6.68124, dim: 4720275, mem: 36Mo 639Ko 816o}

In [11]:
cslbn = csl.CausalModel(bn)

In [12]:
mkv = gum.MarkovBlanket(bn, "X3")
mkv

<pyAgrum.pyAgrum.MarkovBlanket; proxy of <Swig Object of type 'gum::MarkovBlanket *' at 0x79076220afa0> >

In [13]:
csl.doCalculusWithObservation(cslbn, on="Y", doing="T")

$$P( Y \mid \text{do}(T)) = \sum_{V1,V2,V3,X1,X2,X3}{P\left(Y\mid T,V1,V2,V3,X1,X2,X3\right) \cdot P\left(V3\mid X3\right) \cdot P\left(V2\mid X3\right) \cdot P\left(V1\mid X3\right) \cdot P\left(X3\right) \cdot P\left(X2\mid X3\right) \cdot P\left(X1\mid X3\right)}$$

$$\sum_{V1',V2',V3',X1',X2',X3'}{P \left(Y\mid T,V1',V2',V3',X1',X2',X3'\right) \cdot P\left(V3'\mid X3'\right) \cdot P\left(V2'\mid X3'\right) \cdot P\left(V1'\mid X3'\right) \cdot P\left(X3'\right) \cdot P\left(X2'\mid X3'\right) \cdot P\left(X1'\mid X3'\right)}$$

In [14]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [41]:
est = estimator(df, cslbn, "T", "Y", {"X1", "X2", "X3"})

In [61]:
est.Slearner(LinearRegression())

2.011135731102644

In [62]:
est.Tlearner(LinearRegression())

2.0093765126095855

In [63]:
est.Xlearner(LinearRegression())

2.009376512609584

In [68]:
est.Pstrat(1000)

/home/thierry/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


1.99547794274628

In [65]:
est.IPW()

2.0103219406531387

In [66]:
est.AIPW(LinearRegression())

1.9896092485834307